# Risk Model Pipeline - Gerçekçi Gini Testi (%70-80)

Bu notebook:
- Gerçekçi %70-80 Gini aralığında sentetik veri oluşturur
- Pipeline'ın tüm adımlarını test eder
- Filtreleme mekanizmalarının çalıştığını gösterir

## 1. Setup ve Import

In [7]:
pip install pygam

  Using cached pygam-0.10.1-py3-none-any.whl (80 kB)
  Using cached progressbar2-4.5.0-py3-none-any.whl (57 kB)
     --------------------------------------- 46.2/46.2 MB 12.3 MB/s eta 0:00:00
  Using cached python_utils-3.9.1-py2.py3-none-any.whl (32 kB)
     --------------------------------------- 15.9/15.9 MB 13.4 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.5
    Uninstalling numpy-1.21.5:
      Successfully uninstalled numpy-1.21.5
  Attempting uninstall: scipy
    Found existing installation: scipy 1.9.1
    Uninstalling scipy-1.9.1:
      Successfully uninstalled scipy-1.9.1
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.6.0 requires daal==2021.4.0, which is not installed.
numba 0.55.1 requires numpy<1.22,>=1.18, but you have numpy 2.0.2 which is incompatible.


In [8]:
import pandas as pd
import numpy as np
import warnings
import sys
import os
from datetime import datetime
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

# Add parent directory to path
sys.path.append('..')
from src.risk_pipeline.pipeline16 import RiskModelPipeline, Config

warnings.filterwarnings('ignore')
np.random.seed(2024)

print("✅ Setup complete!")

✅ Setup complete!


# Pipeline konfigürasyonu
print("⚙️ Pipeline konfigürasyonu...")

cfg = Config(
    # Temel ayarlar
    id_col='app_id',
    time_col='app_dt',
    target_col='target',
    
    # Veri bölme
    use_test_split=True,
    test_size_row_frac=0.2,
    oot_window_months=4,  # Son 4 ay OOT
    
    # Veri sözlüğü ve kalibrasyon
    data_dictionary_df=data_dict,
    calibration_df=cal_df,
    calibration_method='isotonic',
    
    # Model ayarları
    cv_folds=5,
    random_state=2024,
    n_jobs=2,
    
    # HPO ayarları (hızlı demo için)
    hpo_timeout_sec=20,
    hpo_trials=10,
    
    # Feature engineering eşikleri (DAHA AZ AGRESİF)
    rare_threshold=0.005,     # %0.5'ten az (sadece çok nadir kategoriler)
    psi_threshold=0.30,       # PSI > 0.30 (daha toleranslı)
    iv_min=0.01,             # IV < 0.01 (sadece çok düşük IV elenecek)
    rho_threshold=0.98,      # Korelasyon > 0.98 (neredeyse aynı değişkenler)
    
    # Çıktılar
    output_folder='outputs_realistic_gini',
    output_excel_path='realistic_gini_report.xlsx',
    log_file='outputs_realistic_gini/pipeline.log',
    write_parquet=False,
    write_csv=False
)

print("✅ Konfigürasyon hazır!")
print(f"\n📋 Filtreleme Eşikleri (DAHA AZ AGRESİF):")
print(f"   PSI eşiği: {cfg.psi_threshold} (önceki: 0.20)")
print(f"   IV minimum: {cfg.iv_min} (önceki: 0.02)")
print(f"   Korelasyon eşiği: {cfg.rho_threshold} (önceki: 0.95)")
print(f"   Nadir kategori eşiği: %{cfg.rare_threshold*100} (önceki: %1)")

In [9]:
def create_realistic_credit_data(n_samples=5000):
    """
    Gerçekçi kredi riski verisi - %70-80 Gini hedefli
    
    Özellikler:
    - Güçlü tahmin ediciler (risk_score, payment_score, vb.)
    - Korele değişkenler (korelasyon testi için)
    - Drift eden değişkenler (PSI testi için)
    - Düşük bilgi değerli değişkenler (IV testi için)
    - Nadir kategoriler (rare threshold testi için)
    """
    
    # 1. GÜÇLÜ TAHMİN EDİCİLER (Yüksek IV)
    risk_score = np.random.beta(2, 5, n_samples)  # Ana risk skoru
    payment_score = np.random.beta(3, 7, n_samples)  # Ödeme geçmişi
    debt_burden = np.random.beta(2, 8, n_samples)  # Borç yükü
    income_stability = np.random.beta(4, 6, n_samples)  # Gelir istikrarı
    employment_score = np.random.beta(5, 5, n_samples)  # İstihdam skoru
    
    # 2. KORELE DEĞİŞKENLER (Korelasyon testi için)
    # risk_score ile yüksek korele (>0.95)
    risk_score_v2 = risk_score + np.random.normal(0, 0.05, n_samples)
    risk_score_v2 = np.clip(risk_score_v2, 0, 1)
    
    # payment_score ile korele (>0.90)
    payment_behavior = payment_score * 0.9 + np.random.normal(0, 0.05, n_samples)
    payment_behavior = np.clip(payment_behavior, 0, 1)
    
    # 3. DRIFT EDEN DEĞİŞKEN (PSI testi için)
    time_index = np.arange(n_samples) / n_samples
    drift_feature = np.random.normal(0.3 + 0.4 * time_index, 0.1, n_samples)
    
    # 4. STABIL DEĞİŞKEN (PSI geçecek)
    stable_feature = np.random.normal(0.5, 0.15, n_samples)
    
    # 5. DÜŞÜK BİLGİ DEĞERLİ DEĞİŞKENLER (IV < 0.02, elenecek)
    noise_feature1 = np.random.randn(n_samples)
    noise_feature2 = np.random.uniform(0, 1, n_samples)
    
    # 6. KATEGORİK DEĞİŞKENLER
    # Güçlü kategorik (risk_score ile ilişkili)
    risk_category = pd.cut(risk_score, bins=4, 
                           labels=['Very_Low', 'Low', 'Medium', 'High'])
    
    # Nadir kategoriler içeren (rare threshold testi için)
    product_type = np.random.choice(
        ['Standard', 'Premium', 'Basic', 'Rare1', 'Rare2'], 
        n_samples,
        p=[0.4, 0.3, 0.28, 0.015, 0.005]  # Rare1 ve Rare2 %1'den az
    )
    
    region = np.random.choice(['North', 'South', 'East', 'West'], 
                              n_samples, p=[0.25, 0.25, 0.25, 0.25])
    
    # 7. TARGET OLUŞTURMA (Gerçekçi %70-80 Gini için)
    default_score = (
        3.5 * risk_score +           # En güçlü tahmin edici
        3.0 * payment_score +         # İkinci güçlü
        2.0 * debt_burden +           # Orta güçlü
        1.2 * (1 - income_stability) + # Orta etki
        1.0 * (1 - employment_score) + # Orta etki
        np.random.normal(0, 0.6, n_samples)  # Optimal gürültü
    )
    
    # Target (threshold ile binary)
    threshold = np.percentile(default_score, 50)  # %50 default rate
    target = (default_score > threshold).astype(int)
    
    # 8. DEMOGRAFİK ÖZELLİKLER
    age = np.random.normal(40, 12, n_samples).clip(18, 70)
    tenure = np.random.exponential(5, n_samples).clip(0, 30)
    
    # DataFrame oluştur
    df = pd.DataFrame({
        'app_id': [f'APP_{i:08d}' for i in range(n_samples)],
        'app_dt': pd.date_range('2022-01-01', periods=n_samples, freq='6H'),
        'target': target,
        
        # Güçlü tahmin ediciler
        'risk_score': risk_score,
        'payment_score': payment_score,
        'debt_burden': debt_burden,
        'income_stability': income_stability,
        'employment_score': employment_score,
        
        # Korele değişkenler
        'risk_score_v2': risk_score_v2,
        'payment_behavior': payment_behavior,
        
        # PSI test değişkenleri
        'drift_feature': drift_feature,
        'stable_feature': stable_feature,
        
        # Düşük IV değişkenler
        'noise_feature1': noise_feature1,
        'noise_feature2': noise_feature2,
        
        # Kategorik değişkenler
        'risk_category': risk_category,
        'product_type': product_type,
        'region': region,
        
        # Demografik
        'age': age,
        'tenure': tenure
    })
    
    # Eksik değerler ekle (gerçekçilik için, %2-3)
    missing_cols = ['employment_score', 'tenure']
    for col in missing_cols:
        missing_idx = np.random.choice(df.index, size=int(0.02 * len(df)), replace=False)
        df.loc[missing_idx, col] = np.nan
    
    return df

# Veri oluştur
print("📊 Gerçekçi sentetik veri oluşturuluyor...")
df = create_realistic_credit_data(5000)
print(f"✅ Veri oluşturuldu: {df.shape}")
print(f"   Target oranı: {df['target'].mean():.1%}")
print(f"   Değişken sayısı: {df.shape[1] - 3}")  # id, date, target hariç

📊 Gerçekçi sentetik veri oluşturuluyor...
✅ Veri oluşturuldu: (5000, 19)
   Target oranı: 50.0%
   Değişken sayısı: 16


## 3. Baseline Performans Testi (Logistic Regression)

In [10]:
# Baseline performans testi
print("🔬 Baseline Logistic Regression ile Gini testi...")

# Güçlü değişkenlerle model
strong_features = ['risk_score', 'payment_score', 'debt_burden', 
                  'income_stability', 'employment_score']

# Train/test split
X = df[strong_features].fillna(df[strong_features].median())
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# Logistic Regression
lr = LogisticRegression(random_state=42, max_iter=1000)
lr.fit(X_train, y_train)

# Predictions
y_pred_proba = lr.predict_proba(X_test)[:, 1]

# Performance
auc = roc_auc_score(y_test, y_pred_proba)
gini = 2 * auc - 1

print(f"\n📊 BASELINE PERFORMANS:")
print(f"   AUC: {auc:.3f}")
print(f"   GINI: {gini:.3f} ({gini*100:.1f}%)")

if 0.70 <= gini <= 0.80:
    print(f"\n✅ MÜKEMMEL: Gini hedef aralıkta (%70-80)")
elif gini >= 0.70:
    print(f"\n✅ İYİ: Gini %70+ ({gini*100:.1f}%)")
else:
    print(f"\n⚠️ Gini hedefin altında: {gini*100:.1f}%")

# Feature importance
print("\n📊 Değişken Önemleri (Logistic Regression Coefficients):")
feature_importance = pd.DataFrame({
    'feature': strong_features,
    'coefficient': lr.coef_[0]
}).sort_values('coefficient', ascending=False)
print(feature_importance.to_string(index=False))

🔬 Baseline Logistic Regression ile Gini testi...

📊 BASELINE PERFORMANS:
   AUC: 0.876
   GINI: 0.752 (75.2%)

✅ MÜKEMMEL: Gini hedef aralıkta (%70-80)

📊 Değişken Önemleri (Logistic Regression Coefficients):
         feature  coefficient
      risk_score     8.663711
   payment_score     7.743205
     debt_burden     4.594925
employment_score    -2.212899
income_stability    -2.909652


## 4. Kalibrasyon Verisi ve Veri Sözlüğü

In [11]:
# Kalibrasyon verisi
print("📊 Kalibrasyon verisi hazırlanıyor...")
cal_df = create_realistic_credit_data(1000)
print(f"   Kalibrasyon boyutu: {cal_df.shape}")
print(f"   Kalibrasyon target oranı: {cal_df['target'].mean():.1%}")

# Veri sözlüğü
print("\n📚 Veri sözlüğü hazırlanıyor...")
data_dict = pd.DataFrame({
    'alan_adi': [
        'risk_score', 'payment_score', 'debt_burden', 'income_stability', 
        'employment_score', 'risk_score_v2', 'payment_behavior',
        'drift_feature', 'stable_feature', 'noise_feature1', 'noise_feature2',
        'risk_category', 'product_type', 'region', 'age', 'tenure'
    ],
    'alan_aciklamasi': [
        'Ana risk skoru (0-1, yüksek=riskli)',
        'Ödeme geçmişi skoru (0-1, yüksek=iyi)',
        'Borç yükü göstergesi (0-1, yüksek=borçlu)',
        'Gelir istikrarı (0-1, yüksek=istikrarlı)',
        'İstihdam skoru (0-1, yüksek=güvenli)',
        'Risk skoru v2 (risk_score ile korele, elenecek)',
        'Ödeme davranışı (payment_score ile korele)',
        'Zamanla değişen özellik (PSI ile elenecek)',
        'Sabit özellik (PSI testini geçecek)',
        'Gürültü değişkeni 1 (düşük IV, elenecek)',
        'Gürültü değişkeni 2 (düşük IV, elenecek)',
        'Risk kategorisi (risk_score ile türetilmiş)',
        'Ürün tipi (nadir kategoriler içerir)',
        'Bölge',
        'Müşteri yaşı',
        'Müşteri kıdemi (yıl)'
    ]
})
print(f"   Tanımlanan değişken: {len(data_dict)}")
print("\n📋 Veri Sözlüğü:")
print(data_dict.head(5).to_string(index=False))

📊 Kalibrasyon verisi hazırlanıyor...
   Kalibrasyon boyutu: (1000, 19)
   Kalibrasyon target oranı: 50.0%

📚 Veri sözlüğü hazırlanıyor...
   Tanımlanan değişken: 16

📋 Veri Sözlüğü:
        alan_adi                           alan_aciklamasi
      risk_score       Ana risk skoru (0-1, yüksek=riskli)
   payment_score     Ödeme geçmişi skoru (0-1, yüksek=iyi)
     debt_burden Borç yükü göstergesi (0-1, yüksek=borçlu)
income_stability  Gelir istikrarı (0-1, yüksek=istikrarlı)
employment_score      İstihdam skoru (0-1, yüksek=güvenli)


## 5. Pipeline Konfigürasyonu

In [12]:
# Pipeline konfigürasyonu
print("⚙️ Pipeline konfigürasyonu...")

cfg = Config(
    # Temel ayarlar
    id_col='app_id',
    time_col='app_dt',
    target_col='target',
    
    # Veri bölme
    use_test_split=True,
    test_size_row_frac=0.2,
    oot_window_months=4,  # Son 4 ay OOT
    
    # Veri sözlüğü ve kalibrasyon
    data_dictionary_df=data_dict,
    calibration_df=cal_df,
    calibration_method='isotonic',
    
    # Model ayarları
    cv_folds=5,
    random_state=2024,
    n_jobs=2,
    
    # HPO ayarları (hızlı demo için)
    hpo_timeout_sec=20,
    hpo_trials=10,
    
    # Feature engineering eşikleri
    rare_threshold=0.01,      # %1'den az (Rare1, Rare2 elenecek)
    psi_threshold=0.20,       # PSI > 0.20 (drift_feature elenecek)
    iv_min=0.02,             # IV < 0.02 (noise değişkenler elenecek)
    rho_threshold=0.95,      # Korelasyon > 0.95 (risk_score_v2 elenecek)
    
    # Çıktılar
    output_folder='outputs_realistic_gini',
    output_excel_path='realistic_gini_report.xlsx',
    log_file='outputs_realistic_gini/pipeline.log',
    write_parquet=False,
    write_csv=False
)

print("✅ Konfigürasyon hazır!")
print(f"\n📋 Filtreleme Eşikleri:")
print(f"   PSI eşiği: {cfg.psi_threshold}")
print(f"   IV minimum: {cfg.iv_min}")
print(f"   Korelasyon eşiği: {cfg.rho_threshold}")
print(f"   Nadir kategori eşiği: %{cfg.rare_threshold*100}")

⚙️ Pipeline konfigürasyonu...
✅ Konfigürasyon hazır!

📋 Filtreleme Eşikleri:
   PSI eşiği: 0.2
   IV minimum: 0.02
   Korelasyon eşiği: 0.95
   Nadir kategori eşiği: %1.0


## 6. Pipeline Çalıştırma

In [13]:
# Pipeline çalıştır
print("\n" + "="*60)
print("🚀 PIPELINE ÇALIŞTIRILIYOR...")
print("="*60)

pipeline = RiskModelPipeline(cfg)
pipeline.run(df)

print("\n" + "="*60)
print("✅ PIPELINE TAMAMLANDI!")
print("="*60)


🚀 PIPELINE ÇALIŞTIRILIYOR...
[20:35:51] >> 2) Giris dogrulama & sabitleme basliyor | CPU=6% RAM=44%
[20:35:51] â--  2) Giris dogrulama & sabitleme bitti (0.13s) â€” OK | CPU=2% RAM=44%
[20:35:51] >> 3) Degisken siniflamasi basliyor | CPU=1% RAM=44%
   - numeric=13, categorical=4
[20:35:51] â--  3) Degisken siniflamasi bitti (0.11s) â€” OK | CPU=5% RAM=44%
[20:35:51] >> 4) Eksik & Nadir deger politikasi basliyor | CPU=1% RAM=44%
[20:35:51] â--  4) Eksik & Nadir deger politikasi bitti (0.11s) â€” OK | CPU=1% RAM=44%
[20:35:51] >> 5) Zaman bolmesi (Train/Test/OOT) basliyor | CPU=0% RAM=44%
   - Train=3696, Test=924, OOT=380
[20:35:52] â--  5) Zaman bolmesi (Train/Test/OOT) bitti (0.54s) â€” OK | CPU=1% RAM=44%
[20:35:52] >> 6) WOE binleme (yalniz Train; adaptif) basliyor | CPU=4% RAM=44%
   - WOE hazir: 17 degisken
   - Not: WOE haritasi SADECE TRAIN'de ogrenildi; TEST/OOT icin ayni harita uygulanir (leakage yok)
[20:35:52] â--  6) WOE binleme (yalniz Train; adaptif) bitti (0.20s) â€” OK

## 7. Pipeline Sonuçları ve Analiz

In [14]:
# Sonuçları analiz et
print("\n📊 PIPELINE SONUÇLARI")
print("="*60)

if pipeline.best_model_name_:
    print(f"\n✅ EN İYİ MODEL: {pipeline.best_model_name_}")
    print(f"✅ FİNAL DEĞİŞKEN SAYISI: {len(pipeline.final_vars_)}")
    
    # Final değişkenler
    if pipeline.final_vars_:
        print(f"\n📋 Final Değişkenler:")
        for i, var in enumerate(pipeline.final_vars_, 1):
            desc = data_dict[data_dict['alan_adi'] == var]['alan_aciklamasi'].values
            desc_str = desc[0] if len(desc) > 0 else "Açıklama yok"
            print(f"  {i}. {var}: {desc_str}")
    
    # Model performansı
    if pipeline.models_summary_ is not None and not pipeline.models_summary_.empty:
        best = pipeline.models_summary_[pipeline.models_summary_['model_name'] == pipeline.best_model_name_].iloc[0]
        
        print(f"\n📈 MODEL PERFORMANSI:")
        
        # OOT Performans (en önemli)
        if 'AUC_OOT' in best and best.get('AUC_OOT'):
            auc_oot = best.get('AUC_OOT')
            gini_oot = best.get('Gini_OOT')
            ks_oot = best.get('KS_OOT')
            
            print(f"\n  OUT-OF-TIME (OOT):")
            print(f"    AUC: {auc_oot:.3f}")
            print(f"    GINI: {gini_oot:.3f} ({gini_oot*100:.1f}%)")
            print(f"    KS: {ks_oot:.3f}")
            
            if 0.70 <= gini_oot <= 0.80:
                print(f"\n    🎯 HEDEF BAŞARILI: OOT Gini %70-80 aralığında ({gini_oot*100:.1f}%)")
            elif gini_oot >= 0.70:
                print(f"\n    ✅ İYİ: OOT Gini %70+ ({gini_oot*100:.1f}%)")
            else:
                print(f"\n    ⚠️ OOT Gini hedefin altında: {gini_oot*100:.1f}%")
else:
    print("⚠️ UYARI: Model seçilemedi!")


📊 PIPELINE SONUÇLARI

✅ EN İYİ MODEL: ExtraTrees
✅ FİNAL DEĞİŞKEN SAYISI: 2

📋 Final Değişkenler:
  1. risk_score: Ana risk skoru (0-1, yüksek=riskli)
  2. payment_score: Ödeme geçmişi skoru (0-1, yüksek=iyi)

📈 MODEL PERFORMANSI:

  OUT-OF-TIME (OOT):
    AUC: 0.794
    GINI: 0.588 (58.8%)
    KS: 0.460

    ⚠️ OOT Gini hedefin altında: 58.8%


In [15]:
# Filtreleme detayları
print("\n📊 FİLTRELEME DETAYLARI")
print("="*60)

# PSI analizi
if hasattr(pipeline, 'psi_df_') and pipeline.psi_df_ is not None:
    print("\n🔄 PSI FİLTRESİ:")
    high_psi = pipeline.psi_df_[pipeline.psi_df_['PSI'] > cfg.psi_threshold]
    if not high_psi.empty:
        print(f"  Elenen değişkenler (PSI > {cfg.psi_threshold}):")
        for _, row in high_psi.iterrows():
            print(f"    - {row['variable']}: PSI={row['PSI']:.3f}")
    else:
        print(f"  Tüm değişkenler PSI < {cfg.psi_threshold}")

# Korelasyon analizi
if hasattr(pipeline, 'corr_dropped_') and pipeline.corr_dropped_:
    print("\n🔗 KORELASYON FİLTRESİ:")
    print(f"  Elenen değişkenler (corr > {cfg.rho_threshold}):")
    for item in pipeline.corr_dropped_:
        dropped = item.get('dropped', 'N/A')
        kept = item.get('kept', 'N/A')
        corr = item.get('corr', 0)
        print(f"    - {dropped} elendi (corr={corr:.3f} with {kept})")

# IV analizi
if hasattr(pipeline, 'iv_filter_log_') and pipeline.iv_filter_log_:
    low_iv_vars = [item for item in pipeline.iv_filter_log_ 
                   if item.get('reason', '').startswith('Low IV')]
    if low_iv_vars:
        print("\n📊 IV FİLTRESİ:")
        print(f"  Elenen değişkenler (IV < {cfg.iv_min}):")
        for item in low_iv_vars:
            var = item.get('variable', 'N/A')
            iv = item.get('iv', 0)
            print(f"    - {var}: IV={iv:.4f}")

print("\n🏷️ NADİR KATEGORİ FİLTRESİ:")
print(f"  Eşik: %{cfg.rare_threshold*100}")
print(f"  Muhtemel elenen kategoriler: Rare1, Rare2 (product_type değişkeninde)")


📊 FİLTRELEME DETAYLARI

🏷️ NADİR KATEGORİ FİLTRESİ:
  Eşik: %1.0
  Muhtemel elenen kategoriler: Rare1, Rare2 (product_type değişkeninde)


## 8. Excel Raporu Kontrolü

In [16]:
# Excel raporu kontrolü
import os

excel_path = os.path.join(cfg.output_folder, cfg.output_excel_path)
if os.path.exists(excel_path):
    print(f"📁 Excel raporu oluşturuldu: {excel_path}")
    
    excel_file = pd.ExcelFile(excel_path)
    print(f"\n📋 Sheets ({len(excel_file.sheet_names)} adet):")
    for i, sheet in enumerate(excel_file.sheet_names[:10], 1):  # İlk 10 sheet
        print(f"  {i:2}. {sheet}")
    
    # Model summary
    if 'models_summary' in excel_file.sheet_names:
        models_df = pd.read_excel(excel_path, sheet_name='models_summary')
        print(f"\n📊 MODEL KARŞILAŞTIRMASI:")
        
        # Select available columns
        display_cols = ['model_name', 'Gini_train', 'Gini_test', 'Gini_OOT']
        available_cols = [col for col in display_cols if col in models_df.columns]
        
        if available_cols:
            print(models_df[available_cols].to_string(index=False))
else:
    print(f"⚠️ Excel raporu bulunamadı: {excel_path}")

📁 Excel raporu oluşturuldu: outputs_realistic_gini\realistic_gini_report.xlsx

📋 Sheets (16 adet):
   1. final_vars
   2. best_name
   3. models_summary
   4. best_model
   5. best_model_vars_df
   6. best_model_woe_df
   7. top20_iv_df
   8. top50_univariate
   9. ks_info_traincv
  10. ks_info_test

📊 MODEL KARŞILAŞTIRMASI:
  model_name  Gini_OOT
  ExtraTrees  0.588250
    Logit_L2  0.593798
RandomForest  0.586364
     XGBoost  0.586364
    LightGBM  0.586364
         GAM  0.593798


## 9. Özet ve Sonuçlar

In [17]:
print("\n" + "="*80)
print("📊 TEST ÖZET")
print("="*80)

print("\n✅ TEST EDİLEN PIPELINE ADIMLARI:")
print("  1. Veri validasyon ve hazırlama")
print("  2. WOE binning ve transformation")
print("  3. PSI hesaplama ve filtreleme")
print("  4. IV hesaplama ve düşük bilgili değişkenleri eleme")
print("  5. Korelasyon analizi ve yüksek koreleli değişkenleri eleme")
print("  6. Nadir kategorileri birleştirme")
print("  7. Feature selection (Boruta + Forward Selection)")
print("  8. Model training (6 algoritma)")
print("  9. Model evaluation ve en iyi model seçimi")
print(" 10. Kalibrasyon (isotonic)")
print(" 11. Raporlama ve Excel export")

print("\n📈 HEDEF PERFORMANS:")
print(f"  Hedef Gini aralığı: %70-80")
print(f"  Baseline Gini (Logistic Regression): {gini*100:.1f}%")

if pipeline.best_model_name_ and pipeline.models_summary_ is not None:
    best = pipeline.models_summary_[pipeline.models_summary_['model_name'] == pipeline.best_model_name_].iloc[0]
    if 'Gini_OOT' in best and best.get('Gini_OOT'):
        gini_oot = best.get('Gini_OOT')
        print(f"  Pipeline OOT Gini: {gini_oot*100:.1f}%")
        
        if 0.70 <= gini_oot <= 0.80:
            print("\n🎯 BAŞARILI: Gerçekçi Gini hedefi sağlandı (%70-80)")

print("\n" + "="*80)
print("✅ TEST TAMAMLANDI!")
print("="*80)


📊 TEST ÖZET

✅ TEST EDİLEN PIPELINE ADIMLARI:
  1. Veri validasyon ve hazırlama
  2. WOE binning ve transformation
  3. PSI hesaplama ve filtreleme
  4. IV hesaplama ve düşük bilgili değişkenleri eleme
  5. Korelasyon analizi ve yüksek koreleli değişkenleri eleme
  6. Nadir kategorileri birleştirme
  7. Feature selection (Boruta + Forward Selection)
  8. Model training (6 algoritma)
  9. Model evaluation ve en iyi model seçimi
 10. Kalibrasyon (isotonic)
 11. Raporlama ve Excel export

📈 HEDEF PERFORMANS:
  Hedef Gini aralığı: %70-80
  Baseline Gini (Logistic Regression): 75.2%
  Pipeline OOT Gini: 58.8%

✅ TEST TAMAMLANDI!
